In [ ]:
import coursierapi._
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

In [ ]:
import $ivy.`com.github.dzufferey::scadla:0.1.1`
import scadla._
import InlineOps._
import EverythingIsIn.{millimeters, degrees}
import scadla.utils.thread.ISO
import scadla.utils.CenteredCube
import squants.space.{Length, Millimeters}
import scala.language.postfixOps
import squants.space.LengthConversions._
import scadla.backends.OpenSCAD // for rendering (getting a mesh)
import scadla.backends.almond.Viewer // to show the mesh in jupyter/almond

In [ ]:
import $ivy.`com.github.dzufferey::libgcode:0.1-SNAPSHOT`
import libgcode.utils.{Viewer => NcViewer, _}
import libgcode._
import libgcode.extractor._
import libgcode.generator._
import libgcode.utils.geometry2D._
import scala.collection.mutable.ArrayBuffer

# Code For Structure

## Features (Holes, Slots, etc)

Things that get removed from the base shape

In [ ]:
//this a SubtractiveFeature which gives a model and toolpath
//TODO generalize to multiple plane
abstract class SubtractiveFeature {
    def model(z: Length): Solid
    def toolpath(z: Length)(implicit conf: Config): Seq[Command]
}

case class HoleSpec(
    x: Length,
    y: Length,
    radius: Length,
    depth: Length,
    counterBore: Boolean = false,
    threaded: Boolean = false
) extends SubtractiveFeature {
    
    protected def counterBoreRadius = if (counterBore) 2*radius.to(Millimeters) else 0
    protected def counterBoreDepth = if (counterBore) radius.to(Millimeters) else 0
    protected def _x = x.to(Millimeters)
    protected def _y = y.to(Millimeters)
    protected def _radius = radius.to(Millimeters)
    protected def _depth = depth.to(Millimeters)
    
    def model(z: Length) = {
        val m0 = Cylinder(radius, depth)
        val m1 = Cylinder(counterBoreRadius, counterBoreDepth).moveZ(depth - counterBoreDepth)
        val m2 = m0 + m1
        m2.move(x, y, z - depth)
    }
    
    def spot(z: Length, ignoreCounterBore: Boolean)(implicit conf: Config): Seq[Command] = {
        val _z = z.to(Millimeters)
        if (ignoreCounterBore) {
            Drill(_x, _y, _z, conf.endmillRadius)
        } else {
            Drill(_x, _y, _z - counterBoreDepth, conf.endmillRadius)
        }
    }
    
    def drill(z: Length, ignoreCounterBore: Boolean = false)(implicit conf: Config): Seq[Command] = {
        val _z = z.to(Millimeters)
        if (ignoreCounterBore) {
            Drill(_x, _y, _z, _depth)
        } else {
            Drill(_x, _y, _z - counterBoreDepth, _depth - counterBoreDepth)
        }
    }
    
    protected def threadingHole(radius: Length) = {
        implicit val tolerance = (1e-6 mm)
        if (radius ~= ISO.M1_6) 1.25 / 2
        else if (radius ~= ISO.M2) 0.8
        else if (radius ~= ISO.M2_5) 1
        else if (radius ~= ISO.M3) 1.25
        else if (radius ~= ISO.M4) 1.6
        else if (radius ~= ISO.M5) 2.1
        else if (radius ~= ISO.M6) 2.5
        else if (radius ~= ISO.M8) 3.4
        else if (radius ~= ISO.M10) 4.25
        else if (radius ~= ISO.M12) 5
        else radius.to(Millimeters) * 0.4
    }
    
    def toolpath(z: Length, ignoreShaft: Boolean)(implicit conf: Config): Seq[Command] = {
        val _z = z.to(Millimeters)
        val buffer = scala.collection.mutable.ArrayBuffer.empty[Command]
        if (counterBore) {
            buffer ++= Hole(_x, _y, _z, counterBoreRadius, counterBoreDepth)
        }
        if (!ignoreShaft) {
            var r = if (threaded) threadingHole(radius) else _radius
            buffer ++= Hole(_x, _y, _z - counterBoreDepth, r, _depth - counterBoreDepth)
        }
        buffer.toSeq
    }
    
    def toolpath(z: Length)(implicit conf: Config): Seq[Command] = {
        toolpath(z, false)
    }
}

case class SlotSpec(
    x1: Length, y1: Length,
    x2: Length, y2: Length,
    radius: Length, depth: Length
) extends SubtractiveFeature {
    protected def _x1 = x1.to(Millimeters)
    protected def _y1 = y1.to(Millimeters)
    protected def _x2 = x2.to(Millimeters)
    protected def _y2 = y2.to(Millimeters)
    protected def _radius = radius.to(Millimeters)
    protected def _depth = depth.to(Millimeters)
    
    def model(z: Length): Solid = {
        val _z = z.to(Millimeters)
        val c = Cylinder(_radius,_depth)
        Hull( c.move(_x1,_y1,_z), c.move(_x2,_y2,_z) )
    }
    
    def toolpath(z: Length)(implicit conf: Config): Seq[Command] = {
        val _z = z.to(Millimeters)
        Slot(_x1, _y1, _z, _x2, _y2, _z, _radius, _depth)
    }
}

case class PocketSpec(
    x: Length, y: Length, // lower left corner
    width: Length, length: Length, depth: Length
) extends SubtractiveFeature {
    protected def _x = x.to(Millimeters)
    protected def _y = y.to(Millimeters)
    protected def _width = width.to(Millimeters)
    protected def _length = length.to(Millimeters)
    protected def _depth = depth.to(Millimeters)
    
    def model(z: Length): Solid = {
        Cube(width, length, depth).move(x, y, z-depth)
    }
    
    def toolpath(z: Length)(implicit conf: Config): Seq[Command] = {
        val _z = z.to(Millimeters)
        Rectangle(_x, _y, _z, _width, _length, _depth)
    }
}

## Parts

A base 2D shape with features that gets removed

In [ ]:
object Side extends Enumeration {
  type Side = Value
  val Top, Bottom, Left, Right, Front, Back = Value
}

abstract class Part {
    def model: Solid
    def info: String
    def contour(slotWidth: Length, onionSkin: Length = (0.5 mm))(implicit conf: Config): Seq[Command]
    def chamfer(implicit conf: Config): Seq[Command]
    def spotHoles(ignoreCounterBore: Boolean = false, side: Side.Side = Side.Top)(implicit conf: Config): Seq[Command]
    def drillHoles(ignoreCounterBore: Boolean = false, side: Side.Side = Side.Top)(implicit conf: Config): Seq[Command]
    def boreHoles(ignoreShaft: Boolean = false, side: Side.Side = Side.Top)(implicit conf: Config): Seq[Command]
    def featureToolpath(implicit conf: Config): Seq[Command]
    //TODO print hole info (side, position, diam, depth, threaded)
}

// a part defined by a contour, some holes, and other features where everything is machines from the top
abstract class FlatPart(name: String) extends Part {
    
    // Bounding box for the model
    def width: Length /* X */
    def depth: Length /* Y */
    def height: Length /* Z */

    protected def _width = width.to(Millimeters)
    protected def _depth = depth.to(Millimeters)
    protected def _height = height.to(Millimeters)

    def holes: Seq[(Side.Side,HoleSpec)]
    def otherFeatures: Seq[SubtractiveFeature] = Seq()
    
    def baseModel: Solid
    def baseShape: Path

    def model: Solid = {
        val hs = holes.map{ case (side, hole) =>
            val m = hole.model(0)
            side match {
                case Side.Top => m.moveZ(height)
                case Side.Bottom => m.rotateY(180).moveX(width)
                case Side.Left => m.rotateY(-90)
                case Side.Right => m.rotateY(90).move(width, 0, height)
                case Side.Front => m.rotateX(90)
                case Side.Back => m.rotateX(-90).move(0, depth, height)
            }
        }
        val other = otherFeatures.map( _.model(height) )
        baseModel -- hs -- other
    }
    
    protected def contourLayer(slotWidth: Length)(implicit conf: Config): Path = {
        val base = baseShape
        val sw = slotWidth.to(Millimeters)
        val inner = conf.finishingPass + conf.endmillRadius
        val outer = sw - conf.endmillRadius
        val (nSteps, stepSize) = evenSteps(inner, outer, conf.widthOfCut, conf.roundingError)
        val offsets = (0 until nSteps).map( i => inner + i * stepSize )
        val paths = offsets.map( o => base.offset(-o) ).toSeq
        paths.reduceLeft((p0, p1) => {
            val (a,b) = p0(1.0)
            val (c,d) = p1(0.0)
            val l = Line(a,b,c,d)
            Path(p0.children ++ Seq(l) ++ p1.children)
        })
    }
    
    def contour(slotWidth: Length, onionSkin: Length)(implicit conf: Config): Seq[Command] = {
        val layer = contourLayer(slotWidth)
        val (a,b) = layer(0.0)
        val layerCode = layer.toGCode(conf)
        val (nSteps, stepSize) = evenSteps(_height, onionSkin.to(Millimeters), conf.depthOfCut, conf.roundingError)
        val buffer = scala.collection.mutable.ArrayBuffer.empty[Command]
        // to start
        buffer += G(0, conf.x(a), conf.y(b))
        buffer += G(1, conf.z(_height + conf.travelHeight), F(conf.plungeFeed))
        buffer += Empty(F(conf.feed))
        // layers
        assert(nSteps > 0)
        for (i <- 0 until nSteps - 1) {
            buffer ++= layerCode
            buffer += G(1, conf.x(a), conf.y(b), conf.z(_height + i * stepSize), F(conf.plungeFeed))
            buffer += Empty(F(conf.feed))
        }
        if (conf.finishingPass > 0.0) {
            val path = baseShape.offset(-conf.endmillRadius)
            buffer ++= path.toGCode(conf)
        }
        // to rest
        buffer += G(1, conf.x(a), conf.y(b))
        buffer += G(1, conf.z(_height + conf.travelHeight), F(conf.plungeFeed))
        buffer += Empty(F(conf.feed))
        buffer.toSeq
    }
    
    def chamfer(implicit conf: Config): Seq[Command] = {
        val r = conf.endmillRadius
        val delta = 0.1
        val path0 = baseShape.offset(-delta)
        val path = if (!conf.climb) path0 else path0.flip
        val (a,b) = path(0)
        val c = _height - r + delta
        val buffer = scala.collection.mutable.ArrayBuffer.empty[Command]
        buffer += G(0, conf.x(a), conf.y(b))
        buffer += G(1, conf.z(c), F(conf.plungeFeed))
        buffer += Empty(F(conf.feed))
        buffer ++= path.toGCode(conf)
        buffer += G(1, conf.z(_height + conf.travelHeight), F(conf.plungeFeed))
        buffer += Empty(F(conf.feed))
        buffer.toSeq
    }
        
    def spotHoles(ignoreCounterBore: Boolean, side: Side.Side)(implicit conf: Config): Seq[Command] = side match {
        case Side.Top => holes.flatMap{
            case (Side.Top, h) => h.spot(height, ignoreCounterBore)
            case (_, _) => Seq()
        }
        case _ => ???
    }
    
    def drillHoles(ignoreCounterBore: Boolean, side: Side.Side)(implicit conf: Config): Seq[Command] = side match {
        case Side.Top => holes.flatMap{
            case (Side.Top, h) => h.drill(height, ignoreCounterBore)
            case (_, _) => Seq()
        }
        case _ => ???
    }
    
    def boreHoles(ignoreShaft: Boolean, side: Side.Side)(implicit conf: Config): Seq[Command] = side match {
        case Side.Top => holes.flatMap{
            case (Side.Top, h) => h.toolpath(height, ignoreShaft)
            case (_, _) => Seq()
        }
        case _ => ???
    }
    
    def featureToolpath(implicit conf: Config): Seq[Command] = {
        otherFeatures.flatMap( _.toolpath(height) )
    }
    
    def extraInfo: String = ""
    
    def info = {
        val s = s"""|$name:
                    |  width = $width
                    |  depth = $depth
                    |  height = $height""".stripMargin
        val e = extraInfo
        if (e.length > 0) s + "\n" + e else s
    }
    
}

// a simple part is milled from one part with just a rectangle with some holes
abstract class SimplePart(name: String) extends FlatPart(name) {
    
    def baseModel: Solid = {
        Cube(width, depth, height)
    }
    
    def baseShape: Path = {
        val segments = IndexedSeq(
            Line(0, 0, _width, 0),
            Line(_width, 0, _width, _depth),
            Line(_width, _depth, 0, _depth),
            Line(0, _depth, 0, 0)
        )
        Path(segments)
    }
}

## Assemblies

Put a bunch of parts together

In [ ]:
sealed abstract class Structure {
    def model: Solid
    def parts: Seq[Part]
    def info: String
}

case class SinglePart(part: Part, transform: Solid => Solid = (s: Solid) => s) extends Structure {
    def model = transform(part.model)
    def parts = Seq(part)
    def info = part.info
}

case class Placeholder(dummy: Solid, description: String) extends Structure {
    def model = dummy
    def parts = Seq[Part]()
    def info = description
}

case class Composite(_parts: Seq[(Structure, Seq[Solid => Solid])]) extends Structure {
    def model = Union(_parts.flatMap{ case (p, ts) => ts.map( t => t(p.model) ) }: _*)
    def parts = _parts.flatMap{ case (p, _) => p.parts }
    def info = _parts.map{ case (p, ts) => s"(${ts.size}x) " + p.info }.reduce(_ + "\n" + _)
}

# New Z-axis for my cnc

## Dimensions for mechanical components

In [ ]:
// linear rails (MGN 15C)
object LinearRail {
    val totalHeight = 16 mm // surface to surface distance includes rail + guide
    val width = 15 mm
    val height = 10 mm
    val holesDistance = 40 mm
    val holesRadius = ISO.M3
    def screwPositions(length: Length) = { // x coord of the mounting screw
        val n = ((length - holesRadius * 2) / holesDistance).floor.toInt
        val offset = (length - holesDistance * n) / 2
        (0 until n).map( i => offset + holesDistance * i)
    }
    // extend in x direction [0, length], y centered at 0, back against xy plane
    def rail(length: Length) = {
        val r = Cube(length, width, height).moveY(- width / 2)
        val screw = Cylinder(holesRadius, height) + Cylinder(holesRadius * 2, holesRadius).moveZ(height - holesRadius + 0.1)
        val hs = screwPositions(length).map( x => screw.moveX(x) )
        r -- hs
    }
    def placeholder(l: Length) = {
        new Placeholder(rail(l), s"Linear Rail ($l)")
    }
}
object LinearGuide {
    val length = 42.1 mm
    val width = 32 mm
    val height = 12 mm
    val holesLength = 20 mm
    val holesWidth = 25 mm
    val holesRadius = ISO.M3
    // centered at (0,0), back at 0 and guide facing up
    val holePattern = Seq(( holesLength / 2,  holesWidth / 2),
                          (-holesLength / 2,  holesWidth / 2),
                          ( holesLength / 2, -holesWidth / 2),
                          (-holesLength / 2, -holesWidth / 2))
    val model = {
        val c0 = CenteredCube.xy(length, width, height)
        val c1 = c0 - CenteredCube.xy(length, LinearRail.width, LinearRail.height).moveZ(4)
        c1 -- holePattern.map{ case (x,y) => Cylinder(holesRadius, height).move(x, y, 0) } // make througholes to see align
    }
    def placeholder = {
        new Placeholder(model, "LinearGuide")
    }
}
// ballscrew
object BallScrew {
    // https://www.dold-mechatronik.de/Spindelmuttergehaeuse-DSG1605-fuer-Kugelumlaufspindel-SFU1605-SFU1610
    val nutHeight = 40 mm
    val nutWidth = 52 mm
    val nutLength = 40 mm
    val nutHoles = ISO.M5
    val nutHolesDistanceWidth = 40 mm
    val nutHolesDistanceLength = 24 mm
    val nutHolesPattern = Seq(( nutHolesDistanceLength / 2,  nutHolesDistanceWidth / 2),
                              (-nutHolesDistanceLength / 2,  nutHolesDistanceWidth / 2),
                              ( nutHolesDistanceLength / 2, -nutHolesDistanceWidth / 2),
                              (-nutHolesDistanceLength / 2, -nutHolesDistanceWidth / 2))
    // https://www.dold-mechatronik.de/Festlagereinheit-FLB20-3200-Easy-Mechatronics-System-1620B
    val endHeight = 51 mm
    val endWidth = 70 mm
    val endDepth = 20 mm
    val endHolesDistance = 55 mm
    val endHoles = ISO.M6
    val endHolesPattern = Seq(-endHolesDistance / 2, endHolesDistance / 2)
    val bottomToRotationAxis = 31 mm
    val bearingUnit = {
        val c0 = CenteredCube.xy(endDepth, 40, endHeight)
        val c1 = c0 + CenteredCube.xy(endDepth, endWidth, 10)
        val c2 = c1 -- endHolesPattern.map( y => Cylinder(endHoles, 11).moveY(y) )
        c2
    }
    // length is the distance between mounting screw
    def model(l: Length) = {
        bearingUnit + bearingUnit.moveX(l) + Cylinder(10, l).rotateY(90).moveZ(bottomToRotationAxis)
    }
    def placeholder(l: Length) = {
        new Placeholder(model(l), s"Ball Screw ($l)")
    }
}
// spindle clamp: https://www.dold-mechatronik.de/Eurohalsaufnahme-65x75x20
object SpindleClamp {
    val width = 65 mm
    val height = 20 mm
    val depth = 75 mm
    val backToCenter = 42 mm
    val centerDiameter = 43 mm
    val centerGap = 2 mm
    val mountingHoles = ISO.M8
    val holesPosition = Seq(7.5 mm, 32.5 mm, 57.5 mm)
    val holesHeight = 10 mm
    val model = {
        val x0 = Cube(width, depth, height)
        val x1 = x0 - Cylinder(centerDiameter/2, height).move(width/2, backToCenter, 0)
        val x2 = x1 - CenteredCube.x(centerGap, depth, height).move(width/2, backToCenter + 1, 0)
        val screw = Cylinder(mountingHoles, 17).rotateX(-90)
        val x3 = x2 -- holesPosition.map( x => screw.move(x, 0, height / 2) )
        x3
    }
    def placeholder = {
        new Placeholder(model, "Spindle Clamp")
    }
}

def holesPosition0(n: Int, start: Length, end: Length) = {
    val delta = (end - start) / (n-1)
    (0 until n).map( i => start + delta * i )
}

def holesPosition1(n: Int, length: Length, distanceToBorder: Length) = {
    holesPosition0(n, distanceToBorder, length - distanceToBorder)
}

## XY table (already made)

In [ ]:
object Table extends {
    val mountingHolesWidth = 160 mm
    val mountingHolesDepth = 80 mm
    val mountingHoles = ISO.M8
    val xTravel = 300 mm
    val yTravel = 200 mm
    val distBetweenPlates = BallScrew.nutHeight / 2 +  BallScrew.bottomToRotationAxis
    val plateThickness = 20 mm
    val totalHeigth = plateThickness * 3 + distBetweenPlates * 2
    def model(x: Double = 0.5, y: Double = 0.5) = {
        val lower = CenteredCube.xy(550, 200, plateThickness)
        val mz = plateThickness + distBetweenPlates
        val xPos = xTravel * (0.5 - x)
        val middle = CenteredCube.x(200, 350, plateThickness).move(xPos, -130, mz)
        val uz = mz * 2
        val yPos = yTravel * (1 - y)
        val upper = CenteredCube.xy(300, 200, plateThickness).move(xPos, yPos - 55, uz)
        lower + middle + upper
    }
    val xOffset = 0 mm
    val yOffset = 45 mm
    def placeholder = {
        new Placeholder(model(), "Existing XY Table")
    }
}

In [ ]:
val mesh = OpenSCAD(Table.model())
Viewer(mesh)

## Draft

In [ ]:
val carriageHeight = 150 mm
val pillarInnerspace = BallScrew.nutWidth.max(BallScrew.endWidth) + 10 //TODO smaller endwidth
val expectedTravel = 150 mm

//TODO where to put the z-axis stop

object Pillar extends FlatPart("Pillar") {
    
    val kneeWidth = 50 mm
    val kneeDepth = 20 mm
    val pillarWidth = 60 mm
    
    def width = pillarWidth + kneeWidth
    def depth = Table.totalHeigth +
                 15 /* slots for t-nuts */ +
                 30 + /* spindle collet to collar */
                 carriageHeight + /* carriage heigth */
                 expectedTravel /* expected z travel */
    def height = 15 mm
    
    //TODO rail length should be somewhat standard
    def railLength = depth
    
    val baseHoles = ISO.M8
    val baseHolesNumber = 3
    val baseHolesPositions = {
        val toBorder = baseHoles * 2.5
        val dist = (width - toBorder * 2) / (baseHolesNumber - 1)
        Seq(toBorder, toBorder + dist - 5, toBorder + dist * 2)
    }
    
    val frontThickness = 10 mm
    val frontHole = ISO.M4
    val frontDepth = depth - kneeDepth
    val frontHolesPositions = holesPosition1(6, frontDepth, frontHole * 3 + 10).map( _ + kneeDepth )
    
    def holes = {
        // (1) the base
        val b = baseHolesPositions.map( x => (Side.Front, HoleSpec(x, height / 2, baseHoles, 15, threaded = true) ) )
        // (2) the front
        val f = frontHolesPositions.map( y => (Side.Top, HoleSpec(pillarWidth - frontThickness / 2, y, frontHole, kneeWidth + height, threaded = true) ) )
        // (3) the rails
        val r = LinearRail.screwPositions(railLength).map( y => (Side.Top, HoleSpec(pillarWidth/2, y, LinearRail.holesRadius, height, threaded = true)) )
        b ++ f ++ r
    }
    
    def baseModel = {
        val p = Cube(pillarWidth, depth, height)
        val k = Cube(pillarWidth + kneeWidth, kneeDepth, height)
        p + k
    }
    
    def baseShape = {
        val points = IndexedSeq[(Length,Length)](
            (0, 0),
            (0, width),
            (kneeDepth, width),
            (kneeDepth, pillarWidth),
            (depth, pillarWidth),
            (depth, 0),
            (0, 0)
        )
        val lines = points.sliding(2).map( s => Line(s(0)._1.to(Millimeters),
                                                     s(0)._2.to(Millimeters),
                                                     s(1)._1.to(Millimeters),
                                                     s(1)._2.to(Millimeters)) )
        Path(lines.toIndexedSeq)
    }
    
    override def extraInfo = s"  rail length =  $railLength"

}

In [ ]:
val mesh = OpenSCAD(Pillar.model)
Viewer(mesh)

In [ ]:
object FrontPlate extends SimplePart("Front Plate") {
    
    val width = pillarInnerspace
    val depth = Pillar.frontDepth
    val height = Pillar.frontThickness
    
    val ballScrewLength = expectedTravel + 200 // when I make a lower table, I'll have more travel
    
    def holes = {
        // holes for pillar
        val sideHoles = Pillar.frontHolesPositions.map( y => HoleSpec(height / 2, y, Pillar.frontHole, 10, threaded = true) )
        val left = sideHoles.map( h => (Side.Left, h) )
        val right = sideHoles.map( h => (Side.Right, h) )
        // holes for ballscrew
        def hBS(x: Length, y: Length) = (Side.Top, HoleSpec(x, y, BallScrew.endHoles, height, threaded = true))
        val h2 = depth - 10
        val h3 = h2 - ballScrewLength
        val top = BallScrew.endHolesPattern.map( x => hBS(width/2+x, h2))
        val bot = BallScrew.endHolesPattern.map( x => hBS(width/2+x, h3))
        left ++ right ++ top ++ bot
    }
    
    override def extraInfo = s"  ballScrewLength = $ballScrewLength"
}

In [ ]:
val mesh = OpenSCAD(FrontPlate.model.rotateX(90).moveY(FrontPlate.height))
Viewer(mesh)

In [ ]:
//the model is upside-down
object Base extends SimplePart("Base") {
    def width = 200 mm
    def height = 20 mm
    def depth = (200 mm) /* table bottom plate */ +
                (115 mm) /* table middle plate overhang*/ +
                (20 mm) /* clearance from the top plate */ +
                Pillar.pillarWidth
    // mounting holes for the table (should be threaded!)
    val tableMountingHoles = Seq((width/2 - Table.mountingHolesWidth /2, 100 - Table.mountingHolesDepth / 2),
                                 (width/2 + Table.mountingHolesWidth /2, 100 - Table.mountingHolesDepth / 2),
                                 (width/2 - Table.mountingHolesWidth /2, 100 + Table.mountingHolesDepth / 2),
                                 (width/2 + Table.mountingHolesWidth /2, 100 + Table.mountingHolesDepth / 2))
    // countersunk holes to mount the pillar
    val pillarMountingHoles = {
        val widthDifference = (pillarInnerspace + Pillar.height) / 2
        Pillar.baseHolesPositions.map( y => (width/2 + widthDifference, depth - y) ) ++
        Pillar.baseHolesPositions.map( y => (width/2 - widthDifference, depth - y) )
    }
    
    def holes = {
        tableMountingHoles.map{ case (x,y) => (Side.Top, HoleSpec(x, y, Table.mountingHoles, height, false, true)) } ++
        pillarMountingHoles.map{ case (x,y) => (Side.Top, HoleSpec(x, y, Pillar.baseHoles, height, true, false)) }
    }

}

In [ ]:
object CarriageDimenions {
    val sideThickness = 10 mm
    val frontThickness = 10 mm
    val backThickness = 6 mm
    
    val frontBackClearance = 5 mm
    
    val height = carriageHeight
    val frontHeight = height
    val backHeight = BallScrew.nutLength
    val backHeightOffset = 50 mm
    val innerWidth = pillarInnerspace + Pillar.height * 2 + LinearRail.totalHeight * 2
    val outerWidth = innerWidth + sideThickness * 2
    val sideDepth = Pillar.pillarWidth +
                    frontBackClearance * 2
    val depth = sideDepth +
                frontThickness +
                backThickness
    val upperGuideHeight = LinearGuide.length / 2 + 5
    val lowerGuideHeight = height - upperGuideHeight
    val guideDepth = sideDepth / 2
    
    val frontBackHole = ISO.M4
    val frontHolesPositions = holesPosition1(4, frontHeight, frontBackHole * 3)
    val backHolesPositions = holesPosition1(3, backHeight, frontBackHole * 3)
    
    val cantileverWidth = SpindleClamp.width
    val cantileverDepth = Base.depth -
                          100 /* to middle of table */ -
                          Pillar.pillarWidth / 2 /* to linear rail */ -
                          depth / 2 /* front of carriage */ -
                          sideThickness /* frontplate */ -
                          SpindleClamp.backToCenter -
                          Table.yOffset
    val cantileverHeight = cantileverWidth // + backThickness
    val cantileverHoles = ISO.M5 // or M6 ?
    
    val cantileverHolesPositions = {
        val x = (cantileverWidth - sideThickness) / 2
        val z1 = sideThickness / 2 + 2
        val z3 = cantileverHeight - sideThickness / 2 - 2
        val z2 = (z1 + z3) / 2
        Seq((x,z1),(x,z2),(x,z3),(-x,z1),(-x,z2),(-x,z3))
    }
}

In [ ]:
object CarriageBack extends SimplePart("Carriage Back") {
    import CarriageDimenions._
    
    def width = outerWidth
    def depth = backHeight
    def height = backThickness
    
    def holes = {
        // holes to attach to the ballscrew nut
        BallScrew.nutHolesPattern.map{ case (y,x) => (Side.Top, HoleSpec(width / 2 + x, depth / 2 + y, BallScrew.nutHoles, height, false, true)) } ++
        // holes to attach to side
        backHolesPositions.map( y => (Side.Top, HoleSpec(sideThickness/2, y, frontBackHole, height)) ) ++
        backHolesPositions.map( y => (Side.Top, HoleSpec(width - sideThickness/2, y, frontBackHole, height)) )
    }
}

In [ ]:
object SpindlePlate extends SimplePart("Spindle Plate") {
    import CarriageDimenions._
    
    def width = cantileverWidth
    def depth = cantileverHeight + SpindleClamp.height
    def height = sideThickness
    
    def holes = {
        cantileverHolesPositions.map{ case (x,y) => (Side.Top, HoleSpec(cantileverWidth/2+x, SpindleClamp.height + y, cantileverHoles, height)) } ++
        SpindleClamp.holesPosition.map( x => (Side.Top, HoleSpec(x, SpindleClamp.height/2, SpindleClamp.mountingHoles, height)))
    }
}

In [ ]:
object CantileverTop extends SimplePart("Cantilever Top") {
    import CarriageDimenions._
    
    def width = cantileverWidth
    def depth = cantileverDepth + frontThickness * 2
    def height = backThickness
    
    val topScrew = ISO.M3
    val screwOffset = 20 mm
    
    def holes = {
        val s = sideThickness / 2
        val d = frontThickness + screwOffset
        val hs = Seq(
            HoleSpec(s, d, topScrew, height),
            HoleSpec(width - s, d, topScrew, height),
            HoleSpec(s, depth - d, topScrew, height),
            HoleSpec(width - s, depth - d, topScrew, height)
        )
        hs.map( (Side.Top, _) )
    }
    
    override def otherFeatures: Seq[SubtractiveFeature] = Seq(
        PocketSpec(0, 0, sideThickness, depth, 2),
        PocketSpec(width - sideThickness, 0, sideThickness, depth, 2),
        PocketSpec(0, 0, cantileverWidth, frontThickness, 2),
        PocketSpec(0, depth - frontThickness, cantileverWidth, frontThickness, 2)
    )
}

In [ ]:
// Rotated on the side
object CantileverSide extends SimplePart("Cantilever Side") {
    import CarriageDimenions._

    def width = cantileverHeight
    def depth = cantileverDepth
    def height = sideThickness
    
    def holes = {
        // holes to attach the top
        def topS(x: Length, y: Length) = (Side.Right, HoleSpec(x, y, CantileverTop.topScrew, 10, threaded = true))
        val ts = Seq(topS(sideThickness/2, CantileverTop.screwOffset), topS(sideThickness/2, depth - CantileverTop.screwOffset))
        // holes on the back + front of the sides
        def fbS(x: Length, y: Length, side: Side.Side) = (side, HoleSpec(x, y, cantileverHoles, 15, threaded = true))
        val fs = cantileverHolesPositions.map{ case(x,z) => fbS(z, height/2, Side.Front) }
        val bs = cantileverHolesPositions.map{ case(x,z) => fbS(z, height/2, Side.Back) }
        ts ++ fs ++ bs
    }
    
}

In [ ]:
object CarriageFront extends FlatPart("Carriage Front") {
    import CarriageDimenions._
    
    def width = outerWidth
    def depth = frontHeight
    def height = frontThickness
    
    def holes = {
        def frontBackScrew(x: Length, y: Length) = (Side.Top, HoleSpec(x, y, frontBackHole, height))
        def cantileverScrew(x: Length, y: Length) = (Side.Top, HoleSpec(x, y, cantileverHoles, height))
        // holes to attach to side
        frontHolesPositions.map( y => frontBackScrew(sideThickness/2, y) ) ++
        frontHolesPositions.map( y => frontBackScrew(width - sideThickness/2, y) ) ++
        // holes to attach the cantilever
        cantileverHolesPositions.map{ case (x,y) => cantileverScrew(width/2+x, depth - cantileverHeight + y) }
    }
    
    val kneeWidth = pillarInnerspace + Pillar.height * 2 + 10
    
    def baseModel = {
        val f0 = Cube(width, depth, height)
        // remove material to go over the pillar's knee
        
        val f1 = Cube(kneeWidth, Pillar.kneeDepth + 5, height).moveX((width - kneeWidth) / 2)
        f0 - f1
    }
    
    def baseShape: Path = {
        val wk1 = (width - kneeWidth) / 2
        val wk2 = width - wk1
        val kh = Pillar.kneeDepth + 5
        val points = Seq[(Length, Length)](
            (0, 0),
            (wk1, 0),
            (wk1, kh),
            (wk2, kh),
            (wk2, 0),
            (_width, 0),
            (_width, _depth),
            (0, _depth),
            (0, 0)
        )
        val lines = points.sliding(2).map( s => Line(s(0)._1.to(Millimeters),
                                                     s(0)._2.to(Millimeters),
                                                     s(1)._1.to(Millimeters),
                                                     s(1)._2.to(Millimeters)) )
        Path(lines.toIndexedSeq)
    }

}

In [ ]:
object CarriageSide extends SimplePart("Carriage Side") {
    import CarriageDimenions._
    
    def width = sideDepth
    def depth = CarriageDimenions.height
    def height = sideThickness
    
    def holes = {
        // holes to mount the guides
        def guideScrew(x: Length, y: Length) = (Side.Top, HoleSpec(x, y, LinearGuide.holesRadius, height))
        val hg1 = LinearGuide.holePattern.map{ case (x,y) => guideScrew(guideDepth + y, upperGuideHeight + x) }
        val hg2 = LinearGuide.holePattern.map{ case (x,y) => guideScrew(guideDepth + y, lowerGuideHeight + x) }
        // holes to mount the front
        def frontBackScrew(x: Length, y: Length, side: Side.Side) = (side, HoleSpec(x, y, frontBackHole, 15, threaded = true))
        val f = frontHolesPositions.map( y => frontBackScrew(sideThickness/2, y, Side.Left))
        // holes to mount the back
        val b = backHolesPositions.map( y => frontBackScrew(sideThickness/2, depth - backHeight - backHeightOffset + y, Side.Right) )
        hg1 ++ hg2 ++ f ++ b
    }
}

In [ ]:
object Carriage {
    import CarriageDimenions._

    def side = SinglePart(CarriageSide, _.rotateX(90).rotateZ(90))
        
    def back = SinglePart(CarriageBack, _.rotateX(90).moveY(CarriageBack.height))
    
    def cantilever = {
        // TODO simplify the transforms
        val parts: Seq[(Structure, Seq[Solid => Solid])] = Seq(
            SinglePart(CantileverSide, (s: Solid) => s.rotateY(-90).moveX(CantileverSide.height)) ->
                Seq[Solid => Solid]( s => s.moveX(-cantileverWidth/2), s => s.moveX(cantileverWidth/2 - sideThickness) ),
            SinglePart(CantileverTop, (s: Solid) => s.rotateY(180).move(CantileverTop.width, 0, CantileverTop.height - 2)) ->
                Seq[Solid => Solid]( s => s.move(-cantileverWidth/2, -frontThickness, cantileverHeight) )
        )
        Composite(parts)
    }
    
    def front = SinglePart(CarriageFront, _.rotateX(90).moveY(CarriageFront.height))
    
    def id = Seq[Solid => Solid](s => s)
    
    def sideWithGuides(left: Boolean = true) = {
        val guides = if (left) {
            LinearGuide.placeholder -> Seq[Solid => Solid](
                _.rotateY(-90).rotateZ(180).moveX(sideThickness).move(0, guideDepth, lowerGuideHeight),
                _.rotateY(-90).rotateZ(180).moveX(sideThickness).move(0, guideDepth, upperGuideHeight))
        } else {
            LinearGuide.placeholder -> Seq[Solid => Solid](
                _.rotateY(-90).move(0, guideDepth, lowerGuideHeight),
                _.rotateY(-90).move(0, guideDepth, upperGuideHeight))
        }
        val parts: Seq[(Structure, Seq[Solid => Solid])] = Seq(
            guides,
            side -> id
        )
        Composite(parts)
    }
    
    def spindle = {
        val parts: Seq[(Structure, Seq[Solid => Solid])] = Seq(
            SinglePart(SpindlePlate) -> Seq(_.rotateX(90).moveY(SpindlePlate.height)),
            SpindleClamp.placeholder -> Seq(_.rotateZ(180).moveX(SpindleClamp.width))
        )
        Composite(parts)
    }
    
    def apply() = {
        val parts: Seq[(Structure, Seq[Solid => Solid])] = Seq(
            sideWithGuides(true) -> Seq(_.moveX(-outerWidth/2)),
            sideWithGuides(false) -> Seq(_.moveX(outerWidth/2 - CarriageSide.height)),
            front -> Seq(_.move(-outerWidth/2, -frontThickness, 0)),
            back -> Seq(_.move(-outerWidth/2, sideDepth, height - backHeight - backHeightOffset)),
            cantilever -> Seq(_.move(0, -cantileverDepth -frontThickness, height - cantileverHeight)),
            spindle -> Seq(_.move(-cantileverWidth/2, -cantileverDepth - frontThickness - sideThickness, height - (cantileverHeight + SpindleClamp.height)))
        )
        Composite(parts)
    }
}
//TODO splindle plate should be thicker and have countersunk screw ? or go for some kind of angle brackets / side bar ?
//TODO countersink on the front plate
//TODO save some material on the front plate

In [ ]:
val mesh = OpenSCAD(Carriage().model)
Viewer(mesh)

In [ ]:
OpenSCAD.toSTL(Carriage().model, "carriage.stl")

### Together

In [ ]:
object Cnc {
    
    def base = SinglePart(Base, _.rotateY(180).move(Base.width, 0, Base.height))
    
    def pillar(left: Boolean) = {
        val lt = if(left) {
            Seq[Solid => Solid](_.rotateY(-90).moveY(Pillar.pillarWidth/2))
        } else {
            Seq[Solid => Solid](_.rotateY(-90).rotateZ(180).move(Pillar.height, Pillar.pillarWidth/2, 0))
        }
        Composite(Seq[(Structure, Seq[Solid => Solid])](
            SinglePart(Pillar) -> Seq(_.rotateX(90).rotateZ(-90).move(Pillar.height,Pillar.pillarWidth,0)),
            LinearRail.placeholder(Pillar.railLength) -> lt
        ))
    }
    
    def column = Composite(Seq[(Structure, Seq[Solid => Solid])](
        pillar(true) -> Seq(_.move(-pillarInnerspace / 2 - Pillar.height, 0, 0)),
        pillar(false) -> Seq(_.move(pillarInnerspace / 2, 0, 0)),
        SinglePart(FrontPlate) -> Seq(_.rotateX(90).moveY(FrontPlate.height).move(-FrontPlate.width/2, 0, 0)),
        BallScrew.placeholder(FrontPlate.ballScrewLength) -> Seq(_.rotateY(-90).rotateZ(-90).moveZ(FrontPlate.depth - 10 - FrontPlate.ballScrewLength))
    ))
    
    def apply() = Composite(Seq[(Structure, Seq[Solid => Solid])](
        base -> Seq(_.move(-Base.width / 2, 0 , -Base.height)),
        column -> Seq(_.move(0, Base.depth - Pillar.pillarWidth, 0)),
        Carriage() -> Seq(_.move(0, Base.depth - CarriageDimenions.sideDepth + CarriageDimenions.frontBackClearance, 200)),
        Table.placeholder -> Seq(_.moveY(100))
    ))
}

In [ ]:
val mesh = OpenSCAD(Cnc().model)
Viewer(mesh)

In [ ]:
OpenSCAD.toSTL(Cnc().model, "cnc.stl")

### List of parts

In [ ]:
println(Cnc().info)

## Gearbox to hold the motor and handwheel

For the gear, we will use some files from Thingiverse.
The gears are CC-BY-SA by GeneralRulofDumb, see https://www.thingiverse.com/thing:10955.

The gears (original scale) are offset `12.1 mm` from the intersection of the two rotation axes.

In [ ]:
// get the bevel gear
import java.nio.file.{Paths, Files}
import java.net.{HttpURLConnection,URL}

val gearLeft = FromFile("gear_hypoid_left.stl")
val gearRight = FromFile("gear_hypoid_right.stl")

val gearLeftURL = "https://cdn.thingiverse.com/assets/08/c0/2b/0d/90/gear_hypoid_left.stl"
val gearRightURL = "https://cdn.thingiverse.com/assets/e8/ab/bd/72/48/gear_hypoid_right.stl"

In [ ]:
def getFile(address: String, file: String) = {
    if (!Files.exists(Paths.get(file))) {
        val url = new URL(address)
        val connection = url.openConnection().asInstanceOf[HttpURLConnection]
        connection.connect()
        try {
            val code = connection.getResponseCode
            if (code != 200) {
                println("error " + code)
            } else {
                url #> new File(file) !!
            }
        } finally {
            connection.close()
        }
    }
}

getFile(gearLeftURL, gearLeft)
getFile(gearRightURL, gearRight)

This part is designed to be 3D printed rather than milled.

The faceplate for a nema motor is
- `56.4 mm` wide
- holes are `47.14 mm` apart
- center flange is `38.1mm` in diameter